In [531]:
import es_core_news_sm
from gensim.models import Word2Vec, KeyedVectors
import numpy as np
import regex
import pandas as pd
from scipy.spatial.distance import cosine
import os
import nltk

In [ ]:
nltk.download('popular')

In [532]:
def EntrenarModelo(oraciones,NombreModelo):
    model = Word2Vec(oraciones,vector_size=1250, window=20, min_count=1)
    model.save(NombreModelo)
    
def CargarModelo(NombreModelo):
   modelo = Word2Vec.load(NombreModelo)
   vocabulario = [term for term in modelo.wv.key_to_index]  
   return(modelo,vocabulario)

In [533]:
def ObtenerEmbeddingOracion(modelo, oracion):
   Lista_vectores = []
   for w in nltk.sent_tokenize(oracion, language="spanish"):
       # Verificar que la palabra w exista en el modelo
       try:
           modelo.wv[w]
       except KeyError:
           continue
       # Obtener vector de la palabra
       vec = modelo.wv[w]
       Lista_vectores.append(vec)
   embedding_palabras = np.array(Lista_vectores)
   if (len(embedding_palabras) > 0):
        embedding_oracion = embedding_palabras.mean(axis=0)
   else:
        embedding_oracion = np.zeros(modelo.vector_size)
   return(embedding_oracion) 

In [534]:
def CrearCorpus(path):
  directorio = os.listdir(path)
  corpus = []
  doc_id = []  
  for filename  in directorio:
     texto = open(path+filename,'r',encoding="UTF-8").read()
     corpus.append(texto)
     doc_id.append(filename)
  return(corpus,doc_id)

def PreProcesarOraciones(textos):
    texto_limpio = []
    for texto in textos:
        if len(texto) != 0:
            texto_limpio.append(nltk.sent_tokenize(texto, language="spanish"))
    return texto_limpio

In [535]:
def CrearDiccionario(lista,claves):
   dicc = {}
   for  v in range(0,len(claves)):
      dicc[claves[v]] = lista[v]
   return(dicc)

In [536]:
PATH = "DiscursosOriginales/"
nlp          = es_core_news_sm.load()
corpus,docID = CrearCorpus(PATH)
oraciones = PreProcesarOraciones(corpus)
CorpusConClave  = CrearDiccionario(corpus,docID)


In [537]:
EntrenarModelo(oraciones,'mi_word2vec')

In [538]:
modelo, vocabulario = CargarModelo('mi_word2vec')

In [539]:
doc1 = CorpusConClave['73117.txt']
doc2 = CorpusConClave['136426.txt']
vec1 = ObtenerEmbeddingOracion(modelo, doc1)
vec2 = ObtenerEmbeddingOracion(modelo, doc2)

similitud = 1-cosine(vec2, vec1)
print(similitud)

0.03321518376469612


In [540]:
# Ordenar oraciones por puntaje
def centroide(modelo, oraciones):
    embedding = []
    for oracion in oraciones:
        embedding.append(ObtenerEmbeddingOracion(modelo, oracion))
    return np.mean(embedding, axis=0)
def OrdenarOraciones(oraciones):
    puntaje = []
    centroidee = centroide(modelo, oraciones)
    for oracion in oraciones:
        puntaje.append(cosine(ObtenerEmbeddingOracion(modelo, oracion),centroidee))
    oraciones.sort(key=lambda oracion: cosine(ObtenerEmbeddingOracion(modelo, oracion),centroidee), reverse=True)
    return oraciones

In [541]:
def generar_resumen(O,N,P,U):
    O = OrdenarOraciones(O)
    largo = 1
    Resumen = [""]*(N+1)
    M = len(O)
    for i in range(M):
        if (largo > N): return Resumen[1:N+1]
        Vo = ObtenerEmbeddingOracion(modelo, O[i])
        incluirOracion = False
        for j in range(largo):
            Vr = ObtenerEmbeddingOracion(modelo, Resumen[j])
            Sim = 1-cosine(Vo, Vr)
            if Sim > U and (O[i] not in Resumen):
                incluirOracion = True
        if incluirOracion == True:
            Resumen[largo] = O[i]
            largo+=1
    return Resumen[1:N+1]

In [515]:
text = open('./DiscursosOriginales/72186.txt', encoding='UTF-8').read()
oraciones = PreProcesarOraciones([text])[0]
resumen = generar_resumen(oraciones, 10, 0,0.2)
resumen

/home/parker/Documents/env/jupy/lib64/python3.10/site-packages/scipy/spatial/distance.py:620: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


['Esa entrega que han demostrado a lo largo de la historia de la Fuerza Aérea los hombres y mujeres que componen esta institución, y así lo han demostrado cada vez que la patria los ha necesitado.',
 'Las trágicas muertes de los tenientes Mery, Bello, Vidal, Marsh, el sargento Menadier, el guardiamarina Zañartu, el subteniente Wall y de tantos, fueron las semillas que lograron echar raíces profundas y que han germinado dando grandes frutos en la historia y también en el futuro de nuestra Fuerza Aérea.',
 'Basta con mencionar, y ya lo dijo el Comandante en Jefe, la ayuda que prestaron con ocasión de tantos desastres naturales a lo largo de nuestra historia, los puentes aéreos para llevar alimentos, medicamentos y servicios básicos a miles de compatriotas que, de otra forma, tal vez no habrían logrado sobrevivir.',
 'Y es ese mismo compromiso y entrega que hoy hemos reconocido al entregar la condecoración “General Diego Aracena Aguilar” a seis destacados oficiales y suboficiales y civile